<a href="https://colab.research.google.com/github/Tanmay240405/disease-detection/blob/main/diseaseDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"tanmaysaxena2404","key":"ff86394d4447e0dc917ac4fec905f3ae"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
# =============================
# STEP 1: Upload Kaggle API Key
# =============================
from google.colab import files
files.upload()  # Select kaggle.json

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# =============================
# STEP 2: Download Dataset
# =============================
!kaggle datasets download -d navoneel/brain-mri-images-for-brain-tumor-detection
!unzip -q brain-mri-images-for-brain-tumor-detection.zip -d tumor_dataset

# =============================
# STEP 3: Split Dataset Manually
# =============================
import os, shutil, random

def split_data(source_dir, train_dir, val_dir, split_ratio=0.8):
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)
    classes = os.listdir(source_dir)
    for c in classes:
        class_path = os.path.join(source_dir, c)
        files = os.listdir(class_path)
        random.shuffle(files)
        split_index = int(len(files)*split_ratio)
        train_files = files[:split_index]
        val_files = files[split_index:]

        os.makedirs(os.path.join(train_dir, c), exist_ok=True)
        os.makedirs(os.path.join(val_dir, c), exist_ok=True)

        for f in train_files:
            shutil.copy(os.path.join(class_path, f), os.path.join(train_dir, c, f))
        for f in val_files:
            shutil.copy(os.path.join(class_path, f), os.path.join(val_dir, c, f))

split_data("tumor_dataset", "tumor_dataset/train", "tumor_dataset/val", split_ratio=0.8)

# =============================
# STEP 4: Imports
# =============================
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
import numpy as np
import matplotlib.pyplot as plt

IMG_SIZE = (150, 150)
BATCH_SIZE = 32

# =============================
# STEP 5: Data Generators
# =============================
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    "tumor_dataset/train",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary"
)

val_data = val_datagen.flow_from_directory(
    "tumor_dataset/val",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary"
)

# =============================
# STEP 6: Build Model
# =============================
base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(150,150,3))
base_model.trainable = False  # freeze base completely

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(1, activation="sigmoid")
])

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

# =============================
# STEP 7: Compute Class Weights
# =============================
from sklearn.utils.class_weight import compute_class_weight

classes = train_data.classes
class_weights = compute_class_weight('balanced', classes=np.unique(classes), y=classes)
class_weights = {i: w for i, w in enumerate(class_weights)}
print("Class weights:", class_weights)

# =============================
# STEP 8: Train Model (Top Layers Only)
# =============================
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10,
    class_weight=class_weights
)

# =============================
# STEP 9: Evaluate Model
# =============================
loss, acc = model.evaluate(val_data)
print(f"Validation Accuracy: {acc*100:.2f}%")

plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

# =============================
# STEP 10: Optional Fine-Tuning (Last 10 Layers)
# =============================
base_model.trainable = True
for layer in base_model.layers[:-10]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss="binary_crossentropy",
              metrics=["accuracy"])

history_fine = model.fit(
    train_data,
    validation_data=val_data,
    epochs=3,
    class_weight=class_weights
)

# =============================
# STEP 11: Test on Single Image
# =============================
from tensorflow.keras.preprocessing import image

test_img = "tumor_dataset/val/yes/Y1.jpg"  # replace with your image
img = image.load_img(test_img, target_size=IMG_SIZE)
x = image.img_to_array(img)/255.0
x = np.expand_dims(x, axis=0)

prediction = model.predict(x)[0][0]
print("Prediction:", "Tumor Detected" if prediction>0.5 else "No Tumor Detected")


Saving kaggle.json to kaggle (2).json
Dataset URL: https://www.kaggle.com/datasets/navoneel/brain-mri-images-for-brain-tumor-detection
License(s): copyright-authors
brain-mri-images-for-brain-tumor-detection.zip: Skipping, found more recently modified local copy (use --force to force download)
replace tumor_dataset/brain_tumor_dataset/no/1 no.jpeg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [6]:
!ls tumor_dataset


brain_tumor_dataset  no  yes


In [7]:
!ls tumor_dataset/yes | wc -l
!ls tumor_dataset/no | wc -l


155
98
